In [1]:
from pyspark import SparkContext
import numpy as np
import geopy.distance
import datetime
sc = SparkContext.getOrCreate()


22/04/06 16:10:38 WARN Utils: Your hostname, arch resolves to a loopback address: 127.0.1.1; using 10.50.247.202 instead (on interface wlp4s0)
22/04/06 16:10:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/06 16:10:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/06 16:10:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/04/06 16:10:40 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/04/06 16:10:40 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
def splitAndTake(line):
    arr = line.split('\t')
    lat = float(arr[4])
    lng = float(arr[5])
    mag = float(arr[7])
    date = str(arr[2])
    time = str(arr[3])
    loc = str(arr[-1])
    return (lat,lng,mag,date,time,loc)

In [3]:
dataRDD = sc.textFile('datasets/EarthquakeDataset-Latest.txt')
dataRDD = dataRDD.filter(lambda line : 'Origin Time' not in line)
dataRDD = dataRDD.map(lambda line: splitAndTake(line))
dataRDD = dataRDD.sortBy(lambda x : x[2],ascending=False)

top10quakes = (dataRDD.take(10))

In [11]:
def getAftershocks(quake, line):
    quake_time_str = quake[3] + " " + quake[4]
    line_time_str = line[3] + " " + line[4]
    try:
        quake_time = datetime.datetime.strptime(quake_time_str, "%Y.%m.%d %H:%M:%S.%f") 
    except:
        return false
    try:
        line_time = datetime.datetime.strptime(line_time_str, "%Y.%m.%d %H:%M:%S.%f") 
    except:
        return false
    hour = (line_time - quake_time).total_seconds()/3600
    isAftershock = ((0 < hour) and (hour < 24))
    quake_coord = (quake[0], quake[1])
    line_coord = (line[0], line[1])
    distance = geopy.distance.geodesic(quake_coord, line_coord).km
    isNeighbor = distance <= 20
    #print(distance, isNeighbor, isAftershock, (quake_time - line_time).total_seconds()/3600)
    return (isAftershock and isNeighbor)
    

In [19]:
def getForeshocks(quake, line):
    quake_time_str = quake[3] + " " + quake[4]
    line_time_str = line[3] + " " + line[4]
    try:
        quake_time = datetime.datetime.strptime(quake_time_str, "%Y.%m.%d %H:%M:%S.%f") 
    except:
        return false
    try:
        line_time = datetime.datetime.strptime(line_time_str, "%Y.%m.%d %H:%M:%S.%f") 
    except:
        return false
    hour = (line_time - quake_time).total_seconds()/3600
    isForeshock = ((0 > hour) & (hour > -24))
    quake_coord = (quake[0], quake[1])
    line_coord = (line[0], line[1])
    distance = geopy.distance.geodesic(quake_coord, line_coord).km
    isNeighbor = distance <= 20
    #print(distance, isNeighbor, isForeshock, hour)
    return (isForeshock and isNeighbor)

In [20]:
cnt = 1
for quake in top10quakes:
    print("quake: " ,cnt, " " , quake)
    aftershockRDD = dataRDD.filter(lambda line: getAftershocks(quake, line))
    print("aftershocks: ",aftershockRDD.collect())
    foreshockRDD = dataRDD.filter(lambda line: getForeshocks(quake, line))
    print("foreshocks: ", foreshockRDD.collect())
    print("-------------------------------------")
    cnt += 1

quake:  1   (39.8, 39.51, 7.9, '1939.12.26', '23:57:20.90', 'KURUTILEK- (ERZINCAN) [North East  3.0 km]')


aftershocks:  []


foreshocks:  []
-------------------------------------
quake:  2   (36.54, 27.33, 7.7, '1926.06.26', '19:46:38.80', 'ONIKI ADALAR (AKDENIZ)')


aftershocks:  [(36.5, 27.5, 5.5, '1926.06.26', '21:19:24.00', 'AKDENIZ')]


foreshocks:  []
-------------------------------------
quake:  3   (37.98, 44.48, 7.6, '1930.05.06', '22:34:31.70', 'TÜRKIYE-IRAN SINIR BÖLGESI')


...

foreshocks:  []
-------------------------------------
quake:  9   (39.99, 27.36, 7.2, '1953.03.18', '19:06:16.10', 'SOGUCAK-YENICE (ÇANAKKALE) [South West  2.3 km]')


aftershocks:  [(39.96, 27.59, 5.5, '1953.03.18', '21:18:09.90', 'SOGUTKOY-GONEN (BALIKESIR) [North East  2.8 km]'), (40.0, 27.4, 5.3, '1953.03.18', '20:20:03.00', 'BALLICAY-YENICE (ÇANAKKALE) [North East  0.5 km]'), (40.1, 27.3, 4.9, '1953.03.19', '12:53:04.20', 'ASAGIINOVA-YENICE (ÇANAKKALE) [North East  2.0 km]'), (40.0, 27.4, 4.9, '1953.03.18', '22:28:00.00', 'BALLICAY-YENICE (ÇANAKKALE) [North East  0.5 km]'), (40.0, 27.4, 4.7, '1953.03.18', '23:28:55.00', 'BALLICAY-YENICE (ÇANAKKALE) [North East  0.5 km]')]


foreshocks:  []
-------------------------------------
quake:  10   (35.41, 27.2, 7.2, '1948.02.09', '12:58:18.60', 'AKDENIZ')


aftershocks:  []


foreshocks:  []
-------------------------------------
